In [74]:
import tensorflow as tf
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.examples.tutorials.mnist import input_data

%config InlineBackend.figure_format = 'svg'
learning_rate = 0.01
epochs = 1000
n_samples = 3500
cost_history = np.empty(shape=[1],dtype=float)


# HELPER FUNCTIONS

def get_batch(size, data, target): 
    index = np.arange(0, len(data))
    np.random.shuffle(index)
    index = index[:size]
    batch_data =  [data[ i] for i in index]
    batch_target = [target[ i] for i in index]
    return np.asarray(batch_data), np.asarray(batch_target)

def shuffle(data, target):
    index = np.arange(0, len(data))
    np.random.shuffle(index)
    shuffled_data = [data[i] for i in index]
    shuffled_target = [target[i] for i in index]
    return np.asarray(shuffled_data), np.asarray(shuffled_target)

In [82]:

with np.load("notMNIST.npz") as data :
    Data, Target = data ["images"], data["labels"]
    posClass = 2
    negClass = 9
    dataIndx = (Target==posClass) + (Target==negClass)
    Data = Data[dataIndx]/255.
    Target = Target[dataIndx].reshape(-1, 1)
    Target[Target==posClass] = 1
    Target[Target==negClass] = 0
    np.random.seed(521)
    randIndx = np.arange(len(Data))
    np.random.shuffle(randIndx)
    Data, Target = Data[randIndx], Target[randIndx]
    trainData, trainTarget = Data[:3500], Target[:3500]
    beta = 0.01

    
    validData, validTarget = Data[3500:3600], Target[3500:3600]
    testData, testTarget = Data[3600:], Target[3600:]
    
    trainData_Flat, trainTarget_Flat = trainData.reshape(trainData.shape[0],-1), trainTarget
    validData_Flat, validTarget_Flat = validData.reshape(validData.shape[0],-1), validTarget

    X = tf.placeholder(tf.float32, [None, 784] )
    X_pred = tf.placeholder(tf.float32, [None, 784] )
    Y = tf.placeholder(tf.float32, [None, 1])
    W = tf.Variable(tf.zeros([784, 1]))
    B = tf.Variable(tf.zeros([1]))
    regularizer = tf.nn.l2_loss(W)
    
    
    pred = tf.add(tf.matmul(X,W), B)
    valid_prediction = tf.round(tf.nn.sigmoid(tf.add(tf.matmul(X,W), B)))

    cost = tf.reduce_sum(((tf.pow(pred-Y, 2))/(2*n_samples))+(beta*regularizer))
#     cost = tf.reduce_mean(cost + beta * regularizer)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    
    init = tf.global_variables_initializer()
    
    saver = tf.train.Saver()
    

    with tf.Session() as sess: 
        
        def calc(x, y):
            predictions = tf.round(tf.nn.sigmoid(tf.add(tf.matmul(x,W), B)))
            error = tf.reduce_mean(tf.square(y - predictions))
            return [predictions, error]
        
        sess.run(init)
    
        BATCH_SIZE = 1000
        
        train_count = len(trainData_Flat)    

        for i in range(1, epochs + 1):
            cost_history = np.empty(shape=[1],dtype=float)
            for start, end in zip(range(0, train_count, BATCH_SIZE),
                                  range(BATCH_SIZE, train_count + 1,BATCH_SIZE)):
                sess.run(optimizer, feed_dict={X: trainData_Flat[start:end],
                                           Y: trainTarget_Flat[start:end]})
                cost_history = np.append(cost_history,sess.run(cost,feed_dict={X: trainData_Flat[start:end],Y: trainTarget_Flat[start:end]}))

    
            trainData_Flat, trainTarget_Flat = shuffle(trainData_Flat, trainTarget_Flat)
            
            cost_final = sess.run(cost, feed_dict={X:trainData_Flat, Y:trainTarget_Flat})

        print ("Final MSE COST: ")
        print (cost_final)
        
        predictions = sess.run(valid_prediction, feed_dict={X: validData_Flat})
    
        
        acc, acc_op = tf.metrics.accuracy(labels=validTarget_Flat, 
                                  predictions=predictions)
        
        init_l = tf.local_variables_initializer()
        sess.run(init_l)

        print(sess.run([acc, acc_op]))
        print(sess.run([acc]))
            # Output
            #[0.0, 0.66666669]
            #[0.66666669]
        
        save_path = saver.save(sess, "./linear_reg_model.ckpt")
        print("Model saved in path: %s" % save_path)
          
    
    


Final MSE COST: 
0.07493032
[0.0, 0.44]
[0.44]
Model saved in path: ./linear_reg_model.ckpt
